In [1]:
import os
import numpy as np
import networkx as nx

import process
import pandas

# Chip analysis

The following notebook is an analysis notebook. It has been conceived to order and analyse a dump of confocal images. All questions concerning the code should be emailed to gustave.ronteix@pasteur.fr.

### Processing

The grunt functions are called from process.

Attention: verify the channel order.

In [2]:
livePosition = 2
green = 0
orange = 1

channels = [livePosition, green, orange]

zRatio = 1/3
rNoyau = 9
dCells = 70

path = r'\\atlas.pasteur.fr\Multicell\Gustave\fusionGustave\TIFF BIS'

In [3]:
process._sortFiles(path)

Please verify that the filename order is $t$ then $xy$ then $z$


In [4]:
process._makeSpheroidClass(path, zRatio, rNoyau, dCells, channels)

prep image: 2 folder and time 19
image made, starting nuclei ID
(101, 2044, 2048)
Convolution Time:  154.93708539999997
Coordinates ID Time:  15.91131340000004
44 cells ID
nuclei gotten, make spheroid
refine the analysis over the spheroid
\\atlas.pasteur.fr\Multicell\Gustave\fusionGustave\TIFF BIS\\Spheroids\spheroid_2_19.json
prep image: 2 folder and time 2
image made, starting nuclei ID
(101, 2044, 2048)
Convolution Time:  153.2541777
Coordinates ID Time:  15.850968999999964
135 cells ID
nuclei gotten, make spheroid
refine the analysis over the spheroid
\\atlas.pasteur.fr\Multicell\Gustave\fusionGustave\TIFF BIS\\Spheroids\spheroid_2_2.json
prep image: 2 folder and time 22
image made, starting nuclei ID
(101, 2044, 2048)
Convolution Time:  139.97686799999997
Coordinates ID Time:  20.566528099999914
0 cells ID
nuclei gotten, make spheroid
refine the analysis over the spheroid


ValueError: max() arg is an empty sequence

Remarquons que la segmentation du spheroide dans un ou l'autre des types de cellules depend ici du classificateur naif que nous avons cree. Il repose sur un fit lineaire des points en fonction de leur couleur. On trouve que ce dernier doit avoir comme pente:

$a = \frac{<xy>}{<x^2>}$

Le signe par rapport a celui-ci est donne par:

$sign = y - a*x$

Ce qui nous donne par consequence un moyen rapide pour affecter un genre a chaque cellule.

In [ ]:
IntensityPath = r'\\atlas.pasteur.fr\Multicell\Gustave\fusionGustave\TIFF\Spheroids\\'

In [ ]:
df1 = pandas.read_csv(IntensityPath + 'intensityFrame_0_2.csv')

In [ ]:
import matplotlib.pyplot as plt

ax1 = df1.plot.scatter(x='Orange',y='Green', c= 'r')

In [ ]:
a = df1['Orange'].dot(df1['Green'])/(df1['Orange'].dot(df1['Orange']))

In [ ]:
ax1 = df1.plot.scatter(x='Orange',y='Green', c= 'r')
plt.plot(df1['Orange'], a*df1['Orange'])

In [ ]:
from sklearn import datasets
#Iris Dataset
iris = datasets.load_iris()
X = iris.data

In [ ]:
X = df1[['Orange', 'Green']]

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

#KMeans
km = KMeans(n_clusters=2)
km.fit(X)
km.predict(X)
labels = km.labels_

In [ ]:
df1['newLabels'] = labels

In [ ]:
df1

In [ ]:
from sklearn import mixture
gmm =  mixture.GaussianMixture(n_components=2).fit(X)
labels = gmm.predict(X)
fig, ax = plt.subplots(1, figsize=(7,7))
ax.scatter(X['Orange'], X['Green'],
          c=labels.astype(np.float), edgecolor="k", s=50)
ax.set_xlabel("Petal width")
ax.set_ylabel("Sepal length")
plt.title("Gaussian Mixture", fontsize=14)

### Refined analysis

We use the refined metrics of networkx to add centrality, degree and other measures to the death/survival rates of the spheroid. The following process follows:

 - open json file
 - modify the spheroid properties
 - save global properties in soheroid file
 - save file

In [ ]:
def _refineSph(dic):
    
    G = _makeG(dic)
    
    A = networkx.betweenness_centrality(G) # betweeness centrality
    B = networkx.clustering(G)
    C = networkx.degree(G)
    
    for v in G:
        
        dic['cells'][v]['degree'] = C[v]
        dic['cells'][v]['clustering'] = B[v]
        dic['cells'][v]['centrality'] = A[v]
        
    dic['N'] = len(dic['cells'])
    dic['assortativity'] = nx.degree_assortativity_coefficient(G)
    dic['average degree'] = np.asarray([float(C[v]) for v in G]).mean()
    
    return dic

In [ ]:
_refineSph(_Sph)

process._saveSpheroid(_Sph.Spheroid, path + '\spheroid_' + spheroidFolder + r'_' +  timeFolder + '.json')